# Radar

## 1. Mô tả bài toán

Radar là một công nghệ then chốt trong truyền thông vô tuyến, có nhiều ứng dụng như xe tự hành. Nó thường gồm một anten phát các tín hiệu nhất định và thu lại các tín hiệu phản xạ từ các vật thể xung quanh. Bằng cách xử lý các tín hiệu này, hệ thống có thể xác định phương hướng góc, khoảng cách và vận tốc của các mục tiêu.

Trong ứng dụng thực tế, xử lý tín hiệu radar gặp nhiều khó khăn do nhiễu và các phản xạ từ những vật thể không phải mục tiêu trong môi trường. Ví dụ, khi cố gắng phát hiện người đi bộ, radar có thể đồng thời thu được phản xạ từ cây cối hoặc các vật nền khác, làm giảm độ chính xác. Nhiệm vụ của bạn là dùng AI để phân tích các tín hiệu thu được từ radar và xác định xem tại mỗi vị trí có con người hay không.

Trong bài này chúng tôi cung cấp một **bộ dữ liệu thí nghiệm radar trong nhà**, và mục tiêu của bạn là phát triển một mô hình thực hiện **phân đoạn ngữ nghĩa (semantic segmentation)** trên dữ liệu radar.

## 2. Bộ dữ liệu

Để đo các vật thể xung quanh radar, các tham số chính sau được sử dụng:

* **Range (khoảng cách)**: Khoảng cách thẳng giữa radar và một vật thể.
* **Azimuth (phương vị)**: Góc ngang (trái sang phải) giữa radar và vật thể.
* **Elevation (độ cao / góc chiều dọc)**: Góc thẳng đứng (lên hoặc xuống) của vật thể so với radar.
* **Velocity (vận tốc)**: Tốc độ vật thể đang tiến tới hay lùi khỏi radar.

<img src="figs/Radar Fig 1.png" width="300">

Dữ liệu radar được xử lý thành nhiều **bản đồ nhiệt (heatmaps)**, mỗi bản mã hóa **cường độ tín hiệu nhận được** ở các vị trí và hướng khác nhau.

* **Bản đồ nhiệt tĩnh (static heatmaps)** nhấn mạnh các phản xạ từ các vật thể **đứng yên**.
* **Bản đồ nhiệt động (dynamic heatmaps)** làm nổi bật các thay đổi do **vật thể chuyển động** gây ra.

Khi không có vật thể tại một vị trí cụ thể, tín hiệu chủ yếu là nhiễu nền và xuất hiện yếu. Ngược lại, phản xạ từ một vật thể sẽ làm tăng cường độ tín hiệu, cho phép phát hiện vật thể đó.

Ví dụ, **bản đồ nhiệt khoảng cách–phương vị tĩnh (static range-azimuth heatmap)** thể hiện cường độ tín hiệu theo các khoảng cách (**range**) và các góc ngang (**azimuth**), chủ yếu là phản xạ từ các vật thể tĩnh.

Mỗi mẫu trong bộ dữ liệu được lưu trong file `.mat.pt` dưới dạng tensor có kích thước $7 \times 50 \times 181$, trong đó:

* 7 là số bản đồ (6 bản đồ nhiệt + 1 bản đồ nhãn ngữ nghĩa),
* 50 tương ứng với số bin khoảng cách (range bins),
* 181 tương ứng với số bin góc hoặc vận tốc, bao phủ góc từ -90° đến +90° trong mặt phẳng ngang hoặc dọc. Bạn có thể giả định rằng các bin vận tốc cũng được ánh xạ lại từ -90° đến +90° để tiện trực quan hóa.
* mỗi giá trị cường độ trên bản đồ nhiệt đã được chuẩn hóa về [0, 1], biểu diễn cường độ tín hiệu nhận được.

6 bản đồ nhiệt được cấu trúc như sau:

* **Chỉ số 0**: Bản đồ tĩnh khoảng cách–phương vị (static range-azimuth heatmap)
* **Chỉ số 1**: Bản đồ động khoảng cách–phương vị (dynamic range-azimuth heatmap)
* **Chỉ số 2**: Bản đồ tĩnh khoảng cách–độ cao (static range-elevation heatmap)
* **Chỉ số 3**: Bản đồ động khoảng cách–độ cao (dynamic range-elevation heatmap)
* **Chỉ số 4**: Bản đồ tĩnh khoảng cách–vận tốc (static range-velocity heatmap)
* **Chỉ số 5**: Bản đồ động khoảng cách–vận tốc (dynamic range-velocity heatmap)

Tất cả giá trị trong các bản đồ nhiệt đều **đã được chuẩn hóa**, nên không cần chuyển đổi đơn vị.

**Bản đồ ở Chỉ số 6** là bản đồ nhãn ngữ nghĩa, lưu ở định dạng range-azimuth.

* **-1**: Nền (không có mục tiêu)
* **0**: Va-li (suitcase)
* **1**: Ghế (chair)
* **2**: Người (human)
* **3**: Tường (wall)

Đây là trực quan hóa file `1.mat.pt` trong `training_set`:

<img src="figs/Radar Fig 2.png" width="675">

Một phần mẫu dữ liệu:

<img src="figs/Radar Fig 3.png" width="675">

Quy mô dữ liệu: 1800 mẫu trong tập huấn luyện, 500 mẫu trong tập validation, và 500 mẫu trong tập test.

## 3. Nhiệm vụ

Nhiệm vụ của bạn là phát triển một mô hình nhận **sáu bản đồ nhiệt đầu tiên** (các chỉ số 0 đến 5) làm đầu vào, và dự đoán **bản đồ nhãn ngữ nghĩa** (chỉ số 6) làm đầu ra. Mục tiêu là xác định chính xác nhãn (từ -1 đến 3) tại mỗi vị trí trong trường nhìn của radar.

1. **Input**: Tensor có kích thước $6 \times 50 \times 181$, đại diện cho sáu bản đồ nhiệt radar.
2. **Output**: Tensor có kích thước $50 \times 181$, đại diện cho bản đồ nhãn ngữ nghĩa (label map).

## 4. Nộp bài

Vui lòng nộp một file tên `submission.ipynb`. Kết quả đầu ra là một file zip tên `submission.zip`, chứa hai bảng `submission_val.csv` và `submission_test.csv` tương ứng với kết quả dự đoán trên tập validation và tập test.

**Lưu ý:** Bảng kết quả chỉ cần có header; dữ liệu trong bảng không nhất thiết là dữ liệu đã giải xong, nó chỉ dùng làm ví dụ về định dạng nộp bài.

| filename | pixel_0 | pixel_1 | ... | pixel_9049 |
| :------: | :-----: | ------- | --- | ---------- |
| 1.mat.pt |    -1   | -1      | ... | -1         |
|    ...   |   ...   | ...     | ... | ...        |

## 5. Điểm số

Điểm được tính dựa trên **độ chính xác nhận diện nhãn**. Việc phát hiện đúng các điểm là mục tiêu (non-background) được tính trọng số lớn hơn so với việc nhận diện đúng các điểm nền.

### Tiêu chí chấm:

* Mỗi pixel nền (background) dự đoán đúng được **1 điểm**.
* Mỗi pixel không phải nền (non-background) dự đoán đúng được **50 điểm**.
* Điểm cuối cùng được chuẩn hóa về thang **0–1** bằng cách so sánh với điểm tối đa có thể đạt được.

### Công thức：

$$
Score = \frac{|C_{0,correct}| \times 1 + |C_{1,correct}| \times bonus}{|C_0| \times 1 + |C_1| \times bonus}
$$
với:

$$
\begin{aligned}
I &= {1, 2, \dots, 50\times 181}\
C_0 &= {i \in I \mid y_i = -1}\
C_1 &= {i \in I \mid y_i \neq -1}\
C_{0,correct} &= {i \in C_0 \mid p_i = y_i}\
C_{1,correct} &= {i \in C_1 \mid p_i = y_i}\
\end{aligned}
$$

### Ví dụ

Với bản đồ $3\times3$, giả sử Ground Truth là:

$$
\begin{bmatrix}
-1 & -1 & -1 \
1 & 2 & 3 \
-1 & -1 & -1
\end{bmatrix}
$$

Kết quả mong muốn là:

$$
\begin{bmatrix}
-1 & 1 & -1 \
-1 & 2 & -1 \
-1 & 3 & -1
\end{bmatrix}
$$

Khi đó có bốn pixel `-1` dự đoán đúng và một pixel `2` dự đoán đúng. Điểm của bạn là 4 + 50 = 54. Điểm tối đa có thể là 6 + 50 * 3 = 156 (tức là 6 pixel nền và 3 pixel non-background). Điểm chuẩn hóa là 54 / 156 = 0.346.

$$
Score = \frac{4 \times 1 + 1 \times 50}{6 \times 1 + 3 \times 50}=0.346
$$

## 6. Baseline và Tập huấn luyện

* Dưới đây bạn có thể tìm giải pháp baseline.
* Bộ dữ liệu nằm trong thư mục `training_set`.
* Điểm cao nhất do Ban khoa học đạt được cho bài này là 0.90 trên Leaderboard B; điểm này dùng để chuẩn hóa điểm.
* Điểm baseline do Ban khoa học cho là 0.67 trên Leaderboard B; điểm này cũng dùng để chuẩn hóa điểm.


In [ ]:
import os
data_path = os.getenv("radar_path")
print(data_path)

In [ ]:
import torch
import cv2
import matplotlib.pyplot as plt

tensor = torch.load(f"{data_path}/training_set/1.mat.pt")

In [ ]:
for i in range(tensor.numpy().shape[0]):
    plt.imshow(tensor.numpy()[i])
    plt.show()

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision.models.segmentation import deeplabv3_resnet50
model = deeplabv3_resnet50(weights = None, weights_backbone=None, num_classes=5)
print(model.backbone)


In [ ]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch.nn import Conv2d
from torch import nn
class EfficientNetBackbone(nn.Module):
    def __init__(self):
        super().__init__()
        # Get EfficientNet features only
        backbone = efficientnet_b0(weights=False)
        
        # Modify first conv layer for 6 input channels
        backbone.features[0][0] = Conv2d(6, 32, (3,3), (2,2), (1,1), bias=False)
        
        # Use only features part
        self.features = backbone.features
        
    def forward(self, x):
        # Process features and return dict format expected by DeepLabV3
        x = self.features(x)
        return {"out": x}
model = deeplabv3_resnet50(weights=None, weights_backbone=None, num_classes=5)
model.backbone = EfficientNetBackbone()

In [ ]:
preprocess = EfficientNet_B0_Weights.transforms

In [ ]:
import inspect
import torchvision
functions = inspect.getmembers(torchvision, inspect.isfunction)
for name, func in functions:
    print(name)
# Liệt kê tất cả hàm trong torchvision.utils
import torchvision.utils as utils
functions = inspect.getmembers(utils, inspect.isfunction)

for name, func in functions:
    print(name)

In [ ]:
tensor = torch.load(f"{data_path}/Solution/validation_set/1.mat.pt")

In [ ]:
import pandas as pd
df = pd.read_csv(f"{data_path}/Solution/validation_set/labels/ground_truth_val.csv")
df

In [ ]:
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd

class TrainDataset(Dataset):
    def __init__(self, transform=None):
        super().__init__()
        self.transform = transform
    
    def __len__(self):
        return len(os.listdir(f"{data_path}/training_set"))
    
    def __getitem__(self, idx):
        tensor = torch.load(os.path.join(f"{data_path}/training_set", f"{idx+1}.mat.pt"))
        if self.transform:
            tensor = self.transform(tensor)
        label = tensor[-1] + 1
        return {'data': tensor[:-1].float(), 'label': label.long()}
    
class TestDataset(Dataset):
    def __init__(self, data_path, label_path, transform=None):
        super().__init__()
        self.transform = transform
        self.data_path = data_path
        self.labels = pd.read_csv(label_path)
    def __len__(self):
        files = [file for file in os.listdir(self.data_path) if file.endswith('.pt')]
        return len(files)
    def __getitem__(self, idx):
        data = torch.load(os.path.join(self.data_path, f"{idx+1}.mat.pt"))
        if self.transform:
            data = self.transform(data)
        label_row = self.labels[self.labels['filename'] == f"{idx+1}.mat.pt"]
        if not label_row.empty:
            pixel_columns = [col for col in self.labels.columns if col.startswith('pixel_')]
            label = torch.tensor(label_row[pixel_columns].values[0], dtype=torch.long)
            label = label.reshape(50,181)
            label = label+1

        return {
            'data': data.float(),
            'label': label
        }

In [ ]:
dataset = TrainDataset()
public_test_dataset = TestDataset(f"{data_path}/Solution/validation_set", f"{data_path}/Solution/validation_set/labels/ground_truth_val.csv")
private_test_dataset = TestDataset(f"{data_path}/Solution/test_set", f"{data_path}/Solution/test_set/labels/ground_truth_test.csv")

In [ ]:
from torch.utils.data import random_split
from pytorch_lightning import seed_everything
seed_everything(42, workers=4, verbose=False)
train_size = int(len(dataset) * 0.2)
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = random_split(dataset=dataset, lengths=[train_size, valid_size])

In [ ]:
from torchvision import models, tv_tensors


In [ ]:
train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=36,
                            #   sampler=
                            shuffle=True,
                            num_workers=0
                            )
valid_dataloader = DataLoader(dataset=valid_dataset,
                              batch_size=36,
                            #   sampler=
                            shuffle=True,
                            num_workers=0
                            )

In [ ]:
from torchvision.models import efficientnet_b0
from torch.nn import Conv2d
from torch import nn

class EfficientNetBackbone(nn.Module):
    def __init__(self):
        super().__init__()
        # Get EfficientNet features only
        backbone = efficientnet_b0(weights=False)
        
        # Modify first conv layer for 6 input channels
        backbone.features[0][0] = Conv2d(6, 32, (3,3), (2,2), (1,1), bias=False)
        
        # Use only features part
        self.features = backbone.features
        
        # Add adapter layer to convert 1280 -> 2048 channels
        self.adapter = Conv2d(1280, 2048, kernel_size=1, bias=False)
        
    def forward(self, x):
        # Process features
        x = self.features(x)
        # Convert channels to match DeepLabV3 expectations
        x = self.adapter(x)
        return {"out": x}

model = deeplabv3_resnet50(weights=None, weights_backbone=None, num_classes=5)
model.backbone = EfficientNetBackbone()

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torchvision.ops import sigmoid_focal_loss
import torch.nn.functional as F
from torch import nn
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma
        # ✅ SỬA: Register alpha as buffer để tự động chuyển device
        # self.register_buffer('alpha', torch.tensor([0.05, 1.0, 1.0, 1.0, 1.0]))
    
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        alpha = torch.tensor([len(targets)/len(targets[targets == i]) for i in range(5)], device=inputs.device)
        # ✅ SỬA: alpha đã tự động trên cùng device với inputs
        alpha_t = alpha[targets]
        focal_loss = alpha_t * (1 - pt) ** self.gamma * ce_loss
        
        return focal_loss.mean()

def scoring(y_pred: torch.Tensor, y_true: torch.Tensor):
    y_pred = y_pred.flatten()
    y_true = y_true.flatten()
    
    bg_mask = (y_true == 0)
    object_mask = (y_true != 0)
    
    son = (y_pred[bg_mask] == y_true[bg_mask]).sum() * 1 + (y_pred[object_mask] == y_true[object_mask]).sum() * 50
    mother = (bg_mask.sum()*1 + object_mask.sum() * 50)
    if mother == 0:
        return 0.0
    return (son/mother).float()
class WrapperModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.save_hyperparameters()
        self.model = model
        self.criterion = FocalLoss()
        self.validation_preds = []
        self.validation_labels = []
        self.train_preds = []
        self.train_labels = []

    def forward(self, input):
        return self.model(input)
    
    def training_step(self, batch, idx):
        y_hat = self(batch['data'])['out']
        y_true = batch['label'].to('cuda')
        loss = self.criterion(y_hat, y_true)
        self.log("train_loss", loss, prog_bar=True)
        preds = torch.argmax(y_hat, dim=1).detach().cpu().numpy().flatten()
        targets = y_true.detach().cpu().numpy().flatten()
        self.log("accuracy", accuracy_score(targets, preds))
        self.log("precision", precision_score(targets, preds, average='micro'))
        self.log("recall", recall_score(targets, preds, average='micro'))
        self.log("f1-score", f1_score(targets, preds, average='micro'))
        self.train_preds.append(torch.argmax(y_hat, dim=1).detach().cpu())
        self.train_labels.append(y_true.detach().cpu())
        return loss
    
    def on_train_epoch_end(self):
        all_preds = torch.cat(self.train_preds)
        all_labels = torch.cat(self.train_labels)

        score = scoring(all_preds.flatten(), all_labels.flatten())
        self.log("train_score", score)
        self.train_preds.clear()
        self.train_labels.clear()
    
    def validation_step(self, batch, batch_idx):
        y_hat = self(batch['data'])['out']
        y_true = batch['label']
        loss = self.criterion(y_hat, y_true.to(y_hat.device))
        self.log("val_loss", loss, prog_bar=True)
        preds = torch.argmax(y_hat, dim=1).detach().cpu().numpy().flatten()
        targets = y_true.detach().cpu().numpy().flatten()
        self.log("accuracy", accuracy_score(targets, preds))
        self.log("precision", precision_score(targets, preds, average='micro'))
        self.log("recall", recall_score(targets, preds, average='micro'))
        self.log("f1-score", f1_score(targets, preds, average='micro'))
        self.validation_preds.append(torch.argmax(y_hat, dim=1).detach().cpu())
        self.validation_labels.append(y_true.detach().cpu())
        return loss
    
    def on_validation_epoch_end(self):
        all_preds = torch.cat(self.validation_preds)
        all_labels = torch.cat(self.validation_labels)

        score = scoring(all_preds.flatten(), all_labels.flatten())
        self.log("val_score", score, prog_bar=True)
        cm = confusion_matrix(all_labels.flatten(), all_preds.flatten())
        fig, ax = plt.subplots()
        sns.heatmap(cm, ax=ax, cmap='Blues', annot=cm)
        # ax.set_xticklabels('y_pred')
        # ax.set_yticklabels('y_true')
        ax.set_title(f'Confusion matrix at epoch: {self.current_epoch}')
        self.logger.experiment.add_figure('confusion_matrix', fig, self.current_epoch)
        plt.close(fig=fig)

        self.validation_preds.clear()
        self.validation_labels.clear()

    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=1e-3)
        lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3)
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': lr_scheduler,
                'monitor':'val_loss'
            }
        }
    
trainer = Trainer(
    callbacks=[
        ModelCheckpoint(dirpath='checkpoints/radar_deeplabv3_effb0', filename='best', save_last=True, monitor='val_loss', mode='min'),
        EarlyStopping(mode='min', monitor='val_loss', patience=10)
    ],
    logger=[TensorBoardLogger(save_dir='tb_logs', name='radar_deeplabv3_effb0')],
    max_epochs=1000, gradient_clip_val=1.0, max_time="00:00:15:00", precision=64
)
model = WrapperModel()

In [ ]:
trainer.fit(model, train_dataloader, valid_dataloader)

In [ ]:
private_test_dataloader = DataLoader(dataset=private_test_dataset,
                              batch_size=36,
                            #   sampler=
                            shuffle=False,
                            num_workers=0
                            )
trainer.validate(model, private_test_dataloader)

In [ ]:
public_test_dataloader = DataLoader(dataset=public_test_dataset,
                              batch_size=36,
                            #   sampler=
                            shuffle=False,
                            num_workers=0
                            )
trainer.validate(model, public_test_dataloader)

In [ ]:
import numpy as np
import itertools
from sklearn.metrics import confusion_matrix

y_true = [0,1,2,2,1,0,1,2,0,1]
y_pred = [0,2,1,2,1,0,0,2,0,1]
cm = confusion_matrix(y_true, y_pred)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, None]  # normalize theo hàng

labels = ['class0','class1','class2']

plt.figure(figsize=(6,5))
sns.heatmap(cm_norm, 
            annot=cm, 
            # fmt='d', 
            cmap='Blues',
            # annot_kws={'size':12}, cbar_kws={'format':'%.0f%%'},
            xticklabels=labels, yticklabels=labels)

plt.ylabel('True')
plt.xlabel('Predicted')
plt.title('Confusion matrix (counts + %)')
plt.tight_layout()
plt.show()
